In [407]:
import os
import nipype.pipeline.engine as pe
from nipype.interfaces import fsl

In [408]:
import os
from glob import glob
import pandas as pd
import numpy as np
import nipype.interfaces.afni as afni
import nipype.interfaces.fsl as fsl
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.utility import Function
import seaborn as sns
import nibabel as nb
import json
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe 
import matplotlib.pyplot as plt

os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"


%matplotlib inline

sid = ['001']
base_dir = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/'
work_dir = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/'  
func_dir = os.path.join(base_dir, f'raw_new/sub-{sid[0]}/ses-01/func')
fmap_dir = os.path.join(base_dir, f'raw_new/sub-{sid[0]}/ses-01/fmap')
fsl_dir = os.path.join(base_dir, 'derivatives', 'fsl')  

func_json = sorted(glob(func_dir + '/*.json'))
func_files = sorted(glob(func_dir + '/*.nii.gz'))
fmap_files = sorted(glob(fmap_dir + '/*func*.nii.gz'))

func_files[0]


'/Users/alisha/Desktop/CogNeuro-Imaging-Methods/raw_new/sub-001/ses-01/func/sub-001_ses-01_task-loc1_run-01_bold.nii.gz'

In [409]:
# Here I am building a function that eliminates the
# mapnode directory structure and assists in saving
# all of the outputs into a single directory
def get_subs(func_files):
    '''Produces Name Substitutions for Each Contrast'''
    subs = []
    for curr_run in range(len(func_files)):
        subs.append(('_tshifter%d' %curr_run, ''))
        subs.append(('_volreg%d' %curr_run, ''))
    return subs

# Here I am building a function that takes in a
# text file that includes the number of outliers
# at each volume and then finds which volume (e.g., index)
# has the minimum number of outliers (e.g., min) 
# searching over the first 201 volumes
# If the index function returns a list because there were
# multiple volumes with the same outlier count, pick the first one

# HW3 - change the search window. how does this impact your results?
#I'm not sure how to change the search window but if I narrow the search window
#it might make it more focused but might miss important information outside the search window
#widening the search window provides a wider scope but risks include more artifacts

def best_vol(outlier_count):
    best_vol_num = outlier_count.index(min(outlier_count[:200]))
    if isinstance(best_vol_num, list):
        best_vol_num = best_vol_num[0]
    return best_vol_num

# Here I am creating a list of lists containing the slice timing for each study run
####################################
# HW3 - what column do you query to extract the appropriate information?
#Command F in .json file for 'Slice' 
####################################
slice_timing_list = []
for curr_json in func_json:
    curr_json_data = open(curr_json)
    curr_func_metadata = json.load(curr_json_data)
    slice_timing_list.append(curr_func_metadata["SliceTiming"])

# Here I am establishing a nipype work flow that I will eventually execute
psb6351_wf = pe.Workflow(name='psb6351_wf')
psb6351_wf.base_dir = work_dir + f'/psb6351workdir/sub-{sid[0]}'
psb6351_wf.config['execution']['use_relative_paths'] = True

# Create a Function node to substitute names of files created during pipeline
getsubs = pe.Node(Function(input_names=['func_files'],
                           output_names=['subs'],
                           function=get_subs),
                  name='getsubs')
getsubs.inputs.func_files = func_files

In [410]:
# Here I am inputting just the first run functional data
# FSL instead of AFNI 
# I will use this information to later select the earliest volume with the least number of outliers
# to serve as the base for the motion correction

id_outliers = pe.Node(fsl.MotionOutliers(),
                      name='id_outliers')

# HW3 - what happens if you chose a different func_file?  How does it impact results
# Choosing a different func file would give you different results
# Some runs may be better due to less motion or fewer artifacts

# Is the first func file the best way to go?
# The first run might have more motion because of initial discomfort

id_outliers.inputs.in_file = func_files[0]

# Different for FSL
# Choose metric for FSL 'refrm' as the default 
id_outliers.inputs.metric = 'refrms'
id_outliers.inputs.out_file = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/outliers/outliers.txt'
id_outliers.inputs.in_file = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/raw_new/sub-001/ses-01/func/sub-001_ses-01_task-loc1_run-01_bold.nii.gz'




In [411]:
# Create a Function node to identify the best volume based
# on the number of outliers at each volume. I'm searching
# for the index in the first 201 volumes that has the 
# minimum number of outliers and will use the min() function
# I will use the index function to get the best vol. 
getbestvol = pe.Node(Function(input_names=['outlier_count'],
                              output_names=['best_vol_num'],
                              function=best_vol),
                     name='getbestvol')
psb6351_wf.connect(id_outliers, 'out_file', getbestvol, 'outlier_count')

# Extract the earliest volume with the
# the fewest outliers of the first run as the reference 
extractref = pe.Node(fsl.ExtractROI(t_size=1),
                     name = "extractref")
# HW3 - if you choose a different func file...should this change too?
# Each functional file may have different volumes and characteristics
# Reference volume with the fewest outliers may differ for each run


# If yes, why?  what should it change to?

extractref.inputs.in_file = func_files[0]
psb6351_wf.connect(getbestvol, 'best_vol_num', extractref, 't_min')

In [412]:
# Below is the node that performs motion correction using FSL's MCFLIRT
# this is the node that performs the motion correction
# I'm iterating over the functional files which I am passing
# functional data from the slice timing correction node before
# I'm using the earliest volume with the least number of outliers
# during the first run as the base file to register to.

# HW3 - what happens when you change the order of the following two nodes?
# It would apply motion correction first and then extract the volumes 
# How does that impact your results?
# Reversing might have less accurate corrections because the alignment
# would be based on a volume that has more noise leading to poorer quality 

mcflirt = pe.MapNode(fsl.MCFLIRT(),
                     iterfield=['in_file'],
                     name='mcflirt')
mcflirt.inputs.save_mats = True  
mcflirt.inputs.save_plots = True  
mcflirt.inputs.output_type = 'NIFTI_GZ'
mcflirt.inputs.in_file = func_files
psb6351_wf.connect([
    (extractref, mcflirt, [('roi_file', 'ref_file')]),
])

In [413]:
# Run things again with the order reversed and save in a new sink folder without deleting your first

#mcflirt_first = pe.MapNode(fsl.MCFLIRT(),
                           #iterfield=['in_file'],
                           #name='mcflirt_first')
#mcflirt_first.inputs.save_mats = True
#mcflirt_first.inputs.save_plots = True
#mcflirt_first.inputs.output_type = 'NIFTI_GZ'
#mcflirt_first.inputs.in_file = func_files

#extractref_after = pe.Node(fsl.ExtractROI(t_min=0, t_size=1),
                           #name='extractref_after')

#psb6351_wf_reversed = pe.Workflow(name='psb6351_wf_reversed')
#psb6351_wf_reversed.connect([
    #(mcflirt_first, extractref_after, [('out_file', 'in_file')]),
#])

In [414]:
# Below is the node that runs FSL's slicetimer command
# this is the node that performs the slice timing correction
slicetimer = pe.MapNode(fsl.SliceTimer(),
                        iterfield=['in_file'],
                        name='slicetimer')

slicetimer.inputs.time_repetition = 1.76
slicetimer.inputs.output_type = 'NIFTI_GZ'

# Create a text file with the slice timings
slice_timing_file = os.path.abspath('slice_timings.txt')
np.savetxt(slice_timing_file, np.array(flattened_slice_timing), fmt='%f')

# Use the file path for custom_timings
slicetimer.inputs.custom_timings = slice_timing_file

# Connect the workflow
psb6351_wf.connect([
    (volreg, slicetimer, [('out_file', 'in_file')]),
    # Add other necessary connections here
])

In [415]:
# Below is the node that collects all the data and saves
# the outputs that I am interested in.
datasink = pe.Node(nio.DataSink(), name="datasink")
datasink.inputs.base_directory = output_dir
datasink.inputs.container = f'sub-{sid[0]}'

# Connect the workflow
psb6351_wf.connect([
    (slicetimer, datasink, [('slice_time_corrected_file', 'sltime_corr')]),
    (extractref, datasink, [('roi_file', 'study_ref')]),
    (mcflirt, datasink, [('out_file', 'motion.@corrfile'),
                         ('mat_file', 'motion.@matrix'),
                         ('par_file', 'motion.@par')]),
    (getsubs, datasink, [('subs', 'substitutions')])
])


In [416]:
# The following lines set a work directory outside of my 
# local git repo and runs the workflow
psb6351_wf.run()

241014-01:02:36,370 nipype.workflow INFO:
	 Workflow psb6351_wf settings: ['check', 'execution', 'logging', 'monitoring']
241014-01:02:36,377 nipype.workflow INFO:
	 Running serially.
241014-01:02:36,377 nipype.workflow INFO:
	 [Node] Setting-up "psb6351_wf.id_outliers" in "/Users/alisha/Desktop/CogNeuro-Imaging-Methods/psb6351workdir/sub-001/psb6351_wf/id_outliers".
241014-01:02:36,380 nipype.workflow INFO:
	 [Node] Executing "id_outliers" <nipype.interfaces.fsl.utils.MotionOutliers>
241014-01:02:36,382 nipype.workflow INFO:
	 [Node] Finished "id_outliers", elapsed time 0.000564s.
241014-01:02:36,382 nipype.workflow WARNING:
	 Storing result file without outputs
241014-01:02:36,383 nipype.workflow WARNING:
	 [Node] Error on "psb6351_wf.id_outliers" (/Users/alisha/Desktop/CogNeuro-Imaging-Methods/psb6351workdir/sub-001/psb6351_wf/id_outliers)
241014-01:02:36,384 nipype.workflow ERROR:
	 Node id_outliers failed to run on host Alishas-MacBook-Air.local.
241014-01:02:36,384 nipype.workflo

RuntimeError: 2 raised. Re-raising first.

In [351]:
print(f"Input file path: {func_files[0]}")
print(f"File exists: {os.path.exists(func_files[0])}")

Input file path: /Users/alisha/Desktop/CogNeuro-Imaging-Methods/raw_new/sub-001/ses-01/func/sub-001_ses-01_task-loc1_run-01_bold.nii.gz
File exists: True


In [364]:
import os

input_file = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/raw_new/sub-001/ses-01/func/sub-001_ses-01_task-loc1_run-01_bold.nii.gz'
print("File exists:", os.path.isfile(input_file))

File exists: True


In [253]:
motion_dir = os.path.join(base_dir, f'derivatives/preproc/sub-{sid[0]}/motion')
study_motion_files = sorted(glob(motion_dir + '/*_bold.1D'))

for curr_mot_file in study_motion_files:
    motion_df = pd.read_csv(curr_mot_file, sep="  ", header=None)
    motion_df.columns = ['roll', 'pitch', 'yaw', 'dS', 'dL', 'dP']


    num_vols = range(1, len(motion_df)+1)
    fig, axs = plt.subplots(motion_df.shape[1], 1, figsize = (15, 10))
    # make a little extra space between the subplots
    fig.subplots_adjust(hspace=0.5)

    for idx, curr_col in enumerate(motion_df.keys()):
        axs[idx].plot(num_vols, motion_df[f'{curr_col}'])
        axs[idx].set_xlabel('TRs')
        axs[idx].set_ylabel(f'{curr_col}')
        axs[idx].grid(True)

    plt.show()

####################################
# HW3 - compare the motion parameter plots from this script to 
# the carpet plot below.  What is the relationship between large rotations and/or translations and the carpet plot
# reference specific TRs and their relation to time.
####################################